# SQL ALCHEMY TUTORIAL

In [1]:
""" main.py """
import os
from pathlib import Path

from sqlalchemy import create_engine, event, Engine
from sqlalchemy.orm import scoped_session, sessionmaker
from models import Model

BASE_DIR = Path('.').absolute()
engine = create_engine(f"sqlite:///{BASE_DIR}/db.sqlite3", echo=False)
session = scoped_session(
    sessionmaker(
        autoflush = False,
        autocommit = False,
        bind=engine
    )
)

@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

In [2]:
""" models.py """
from datetime import datetime

from sqlalchemy import Column, Integer, String, Boolean, ForeignKey, DateTime, Date, Float
from sqlalchemy.orm import Relationship
from sqlalchemy.orm import declarative_base
from main import session

Model = declarative_base()
Model.query = session.query_property()


class User(Model):
    __tablename__ = "user"
    id = Column(Integer, primary_key=True, autoincrement=True)

    created_at = Column(DateTime, default=datetime.now())
    last_updated = Column(DateTime, onupdate=datetime.now())
    name = Column(String, nullable=False)
    age = Column(Integer, nullable=True)
    # uselist ensures you get a single value when querying instead of a list, which would be the case for on to many relatiohsips
    author_profile = Relationship("AuthorProfile", 
                                  back_populates="user", 
                                  passive_deletes=True, 
                                  uselist=False)


class AuthorProfile(Model):
    __tablename__ = "profile"
    id = Column(Integer, primary_key=True, autoincrement=True)
    # unique forces one to one relationship
    user_id = Column(Integer, ForeignKey("user.id", ondelete='CASCADE'), unique=True, nullable=True)
    display_name = Column(String, nullable=False)
    bio = Column(String, nullable=True)
    
    user = Relationship("User", back_populates="author_profile")
    # many to many with blog
    blogs = Relationship("Blog",
                         secondary="blog_author",
                         passive_deletes=True,
                         back_populates="authors"
                        )


class BlogAuthor(Model):
    __tablename__ = "blog_author"
    id = Column(Integer, primary_key=True, autoincrement=True)
    author_profile_id = Column(Integer, ForeignKey("profile.id", ondelete='CASCADE'), nullable=True)
    blog_id = Column(Integer, ForeignKey("blog.id"), nullable=True)


class Blog(Model):
    __tablename__ = "blog"
    id = Column(Integer, primary_key=True, autoincrement=True)
    
    created_at = Column(DateTime, default=datetime.now())
    last_updated = Column(DateTime, onupdate=datetime.now())
    authors = Relationship("AuthorProfile",
                         secondary="blog_author",
                         passive_deletes=True,
                         back_populates="blogs")
    title = Column(String, nullable=False)
    tagline = Column(String, nullable=True)
    content = Column(String, nullable=False)
    comments = Relationship("Comment")


class Comment(Model):
    __tablename__ = "comment"
    id = Column(Integer, primary_key=True, autoincrement=True)
    blog_id = Column(Integer, ForeignKey("blog.id", ondelete='CASCADE'), nullable=True)
    text = Column(String, nullable=False)
    blog = Relationship("Blog",
                         passive_deletes=True,
                         back_populates="comments",
                         uselist=False)

`!sqlite3 db.sqlite3`
```
>> SELECT * FROM user;
```

In [3]:
Model.metadata.create_all(engine)

- In terminal run:
`python3 seed.py`

In [4]:
""" data snippet """
users = [
    {
        "id": 1,
        "name": "John Dillons",
        "age": 25,
    },
    {
        "id": 2,
        "name": "James Smith",
        "age": 25,
    },
    {
        "id": 3,
        "name": "Alex Shmoe",
        "age": 25,
        
    },
    {
        "id": 4,
        "name": "Rob Scramble",
        "age": 25,
        
    },
    {
        "id": 5,
        "name": "Maxwell Amadeous",
        "age": 25,
        
    }
]

authors = [
    {
        "id": 1,
        "display_name": "Ashmoe28",
        "user_id": 3
    },
    {
        "id": 2,
        "display_name": "Mamadeous7",
        "user_id": 5
    },
    {
        "id": 3,
        "display_name": "JSmith11",
        "user_id": 2,
    },
]

blogs = [
    {
        "id": 1,
        "title": "Beginner Django Tutorial",
        "tagline": "How to get started with django",
        "content": "Mollit culpa aute labore dolore quis eu pariatur esse consequat. Excepteur proident sint aliqua pariatur cupidatat exercitation non anim consectetur. Irure magna mollit irure velit sint veniam enim. Ullamco culpa laboris ullamco cillum tempor sit Lorem non. Mollit voluptate magna laborum ipsum. Adipisicing amet ipsum minim eiusmod exercitation irure duis culpa labore proident. Magna laborum id reprehenderit enim laborum exercitation eu."
    },
    {
        "id": 2,
        "title": "Beginner Flask Tutorial",
        "tagline": "How to get started with flask",
        "content": "Mollit culpa aute labore dolore quis eu pariatur esse consequat. Excepteur proident sint aliqua pariatur cupidatat exercitation non anim consectetur. Irure magna mollit irure velit sint veniam enim. Ullamco culpa laboris ullamco cillum tempor sit Lorem non. Mollit voluptate magna laborum ipsum. Adipisicing amet ipsum minim eiusmod exercitation irure duis culpa labore proident. Magna laborum id reprehenderit enim laborum exercitation eu."
    },
    {
        "id": 3,
        "title": "Machine Learning Tutorial",
        "tagline": "How to get started with machine learning with python",
        "content": "Mollit culpa aute labore dolore quis eu pariatur esse consequat. Excepteur proident sint aliqua pariatur cupidatat exercitation non anim consectetur. Irure magna mollit irure velit sint veniam enim. Ullamco culpa laboris ullamco cillum tempor sit Lorem non. Mollit voluptate magna laborum ipsum. Adipisicing amet ipsum minim eiusmod exercitation irure duis culpa labore proident. Magna laborum id reprehenderit enim laborum exercitation eu."
    },
    {
        "id": 4,
        "title": "SQL Tutorial",
        "tagline": "Querying databases with SQL",
        "content": "Mollit culpa aute labore dolore quis eu pariatur esse consequat. Excepteur proident sint aliqua pariatur cupidatat exercitation non anim consectetur. Irure magna mollit irure velit sint veniam enim. Ullamco culpa laboris ullamco cillum tempor sit Lorem non. Mollit voluptate magna laborum ipsum. Adipisicing amet ipsum minim eiusmod exercitation irure duis culpa labore proident. Magna laborum id reprehenderit enim laborum exercitation eu."
    },
]

blog_authors = [
    {
        "id": 1,
        "author_profile_id": 2,
        "blog_id": 1, 
    },
    {
        "id": 2,
        "author_profile_id": 3,
        "blog_id": 4, 
    },
    {
        "id": 3,
        "author_profile_id": 1,
        "blog_id": 2, 
    },
    # {
    #     "id": 4,
    #     "author_profile_id": 3,
    #     "blog_id": 4, 
    # },
]

comments = [
    {
        "id": 1,
        "blog_id": 1,
        "text": "Love it!"

    },
    {
        "id": 2,
        "blog_id": 1,
        "text": "Can't wait for more!"

    },
    {
        "id": 3,
        "blog_id": 2,
        "text": "Can't wait for your next tutorial!"

    },
    {
        "id": 4,
        "blog_id": 2,
        "text": "Love it!"

    },
    {
        "id": 5,
        "blog_id": 2,
        "text": "Great Post!"

    },
    {
        "id": 6,
        "blog_id": 3,
        "text": "Not really a fan of this one!"

    },
    {
        "id": 7,
        "blog_id": 4,
        "text": "Very good post!"

    },
]

In [5]:
def seed_db():
    data = [(users, User), (authors, AuthorProfile), (blogs, Blog), 
            (blog_authors, BlogAuthor), (comments, Comment)]
    for objects, model in data:
        for obj in objects:
            session.add(model(**obj))
        session.commit()

seed_db()

## Querying

### SELECT

In [6]:
def select_all_blogs():
    blogs = Blog.query.all()
    for blog in blogs:
        print(blog.title)



In [8]:
select_all_blogs()

Beginner Django Tutorial
Beginner Flask Tutorial
Machine Learning Tutorial
SQL Tutorial


In [10]:
def print_blogs(q):
    for blog in q: 
        print(blog.title)
        print(blog.tagline)
        print(blog.created_at)

In [11]:
def filter_blogs():
    q_1 = Blog.query.filter(Blog.title == "Beginner Django Tutorial")
    q_2 = Blog.query.filter_by(title="Beginner Django Tutorial")
    q_3 = Blog.query.filter(Blog.title.contains("Beginner"))
    print_blogs(q_3)


In [12]:
filter_blogs()

Beginner Django Tutorial
How to get started with django
2024-06-07 15:06:11.886177
Beginner Flask Tutorial
How to get started with flask
2024-06-07 15:06:11.886177


### Insert

In [13]:
u = User(name="New User", age=55)
print(f"BEFORE: {u.id}")
session.add(u)
session.commit()
print(f"AFTER: {u.id}")


BEFORE: None
AFTER: 6


In [14]:
def print_users(u):
    for user in u:
        print(user.name, user.age)    

def insert_users():
    u = User(name="Clark Kent", age=37)
    u2 = User(name="Clark Kent", age=37)
    session.add(u)
    session.add(u2)
    session.commit()

    users = User.query.filter(User.name.contains("Clark"))
    print_users(users)


In [15]:
insert_users()

Clark Kent 37
Clark Kent 37


### Update

In [16]:
def update_user():
    u = User.query.filter_by(name="Clark Kent").first()
    u.name = "Superman"
    session.commit()
    

In [17]:
update_user()

### Delete

In [18]:
deleted = User.query.filter_by(name="Clark Kent").delete() 
deleted

1

### joins

In [19]:
# 1 to 1 example
def get_user_profiles():
    """ one to one example with user and profiles """
    profiles = AuthorProfile.query.join(User, AuthorProfile.user)
    print(f"DISPLAY_NAME: {profiles.first().display_name}")
    print(f"NAME: {profiles.first().user.name}")

In [20]:

# 1 to many example
def get_blog_comments():
    """ One to many example: grabbing all comments asssociated with one blog """
    blogs = Blog.query.join(Comment, Blog.comments).all()
    for comment in blogs[1].comments:
        print(comment.text)

In [22]:
get_blog_comments()

Can't wait for your next tutorial!
Love it!
Great Post!


### Groupby and aggregation functions

In [23]:
from sqlalchemy import or_, and_, not_, desc, asc, select, func
from sqlalchemy.orm import load_only

In [24]:
# grab the 
res = session.query(User.id, User.name, func.max(User.age)).all()
res

[(6, 'New User', 55)]

In [25]:
from sqlalchemy import func
res = User.query.with_entities(func.max(User.age), User.name, User.id).all()
res

[(55, 'New User', 6)]

In [26]:
def get_comment_counts():
    num_comments_by_blog = Comment.query \
        .with_entities(Comment.blog_id, 
                        func.count(Comment.blog_id)) \
        .group_by(Comment.blog_id).all()
    print(num_comments_by_blog)

get_comment_counts()

[(1, 2), (2, 3), (3, 1), (4, 1)]
